In [1]:
import time
import os
import copy

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms

# Pennylane
import pennylane as qml
from pennylane import numpy as np

torch.manual_seed(42)
np.random.seed(42)

# Plotting
import matplotlib.pyplot as plt

# OpenMP: number of parallel threads.
os.environ["OMP_NUM_THREADS"] = "1"

/home/ram/Anaconda/anaconda3/envs/covalent/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchvision import datasets, transforms, models
from torch import optim
from torch.utils.data import DataLoader


import numpy as np
import pandas as pd
import os
import scipy.ndimage as ndi
import shutil

In [3]:
import covalent as ct

In [4]:
n_qubits = 5
dev = qml.device("lightning.qubit", wires=n_qubits)

In [5]:
@ct.electron
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)

@ct.electron
def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)

@ct.electron
def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT.
    """
    # In other words it should apply something like :
    # CNOT  CNOT  CNOT  CNOT...  CNOT
    #   CNOT  CNOT  CNOT...  CNOT
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

In [6]:
@ct.electron
@qml.qnode(dev, interface="torch",diff_method="adjoint")
def quantum_net(q_input_features, q_weights_flat):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    RY_layer(q_input_features)

    # # # Sequence of trainable variational layers
    for k in range(q_depth):
        qml.BasicEntanglerLayers(weights=q_weights, wires=range(n_qubits))
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)

In [7]:
q_depth = 1                 # Depth of the quantum circuit (number of variational layers)
gamma_lr_scheduler = 0.1    # Learning rate reduction applied every 10 epochs.
q_delta = 0.01              # Initial spread of random quantum weights
start_time = time.time()    # Start of the computation timer

In [8]:
@ct.electron
class DressedQuantumNet(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()
        self.pre_net = nn.Linear(512, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.zeros(q_depth * n_qubits)) # 2 for QAOA //rand
        self.post_net = nn.Linear(n_qubits, 2)
        self.relu = torch.nn.ReLU()

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """

        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 512 to 4
        pre_out = self.pre_net(input_features)
        q_in = self.relu(pre_out) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = quantum_net(elem, self.q_params).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_hybrid = models.resnet18(pretrained=True)
model_hybrid.fc = DressedQuantumNet()
model_hybrid = model_hybrid.to(device)

In [10]:
model_hybrid.load_state_dict(torch.load('./model_14.pt',map_location=torch.device('cpu'))['model_state_dict'])

<All keys matched successfully>

In [11]:
@ct.electron
def evaluation(dataloader,model):
    arr = []
    with torch.no_grad():
        total,correct = 0,0
        for data in dataloader:
            inputs,labels = data
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = model(inputs)
            _,pred = torch.max(outputs.data,1) 
             # pred = pred + 1
            print(pred)
            total += labels.size(0)
            correct += (pred==labels).sum().item()
            arr.append(pred)

        return arr

In [12]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from PIL import Image 
#import skimage.transform as skTrans
@ct.electron
class CancerClass(Dataset):
    def __init__(self,csv_data_x,csv_data_y,root_dir,transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.datax = csv_data_x
        self.datay = csv_data_y

    def __len__(self):
        return len(self.datax)

    def __getitem__(self,idx):
        filename = str(self.datax[idx])
        img_path = self.root_dir +'/'+ filename + '.tif'

        image_array = Image.open(img_path)
        image_array1 = image_array

        del image_array

        label = int(self.datay[idx])

        if self.transform:
            tensor_image = self.transform(image_array1)

        return tensor_image,label

In [13]:
val_data_new_x = ['0000d563d5cfafc4e68acb7c9829258a298d9b6a','0000da768d06b879e5754c43e2298ce48726f722','0000f8a4da4c286eee5cf1b0d2ab82f979989f7b','000a2a35668f04edebc0b06d5d133ad90c93a044','000aa5d8f68dc1f45ebba53b8f159aae80e06072','000aa7c34dc319d936d36f7f4c257812d3d03cdf','000aa638312a3dad22ef04b8a7df3fc98fc2e7c3','000af35befdd9ab2e24fac80fb6508dfd1edd172','000b35e7c39c6cb32224dcb3fe4c48acf34f0252','000b666f7b5f03e81937cb12b3a1c8c279b08292','000c3d0468a1a0a7a35a2e453f4b891b3a4e7fb6','000c4f225ec3f4d1eaa986d75596cc71e10568b6','000d3de1f31201b54cf82572c10099606f33c791','000d4bcc9d239e8304890ffd764794e93504e475']
val_data_new_y = [0,1,0,1,1,1,0,1,1,0,1,0,1,0]

In [14]:
batch_size = 1
val_set = CancerClass(val_data_new_x,val_data_new_y,'/home/ram/new_temp/Images',transforms.Compose(
    [transforms.ToTensor()]))
val_loader = torch.utils.data.DataLoader(val_set,batch_size=batch_size,shuffle=False)


In [15]:
@ct.lattice
def workflow(model,dataloader):
    val = evaluation(dataloader,model)
    return val

In [16]:
dispatch_id = ct.dispatch(workflow)(model_hybrid, val_loader)
result = ct.get_result(dispatch_id=dispatch_id, wait=True)
val = result.result
print(val)

[tensor([1]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([1]), tensor([1]), tensor([0]), tensor([1]), tensor([1]), tensor([0]), tensor([0])]


In [17]:
evaluation(val_loader,model_hybrid)

tensor([1])
tensor([0])
tensor([1])
tensor([0])
tensor([0])
tensor([0])
tensor([0])
tensor([1])
tensor([1])
tensor([0])
tensor([1])
tensor([1])
tensor([0])
tensor([0])


[tensor([1]),
 tensor([0]),
 tensor([1]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([0]),
 tensor([1]),
 tensor([1]),
 tensor([0]),
 tensor([1]),
 tensor([1]),
 tensor([0]),
 tensor([0])]